install required packages

In [1]:
%pip install --upgrade --quiet  langchain langchain-community langchain-ollama langchain-experimental neo4j tiktoken yfiles_jupyter_graphs python-dotenv json-repair langchain-openai langchain_core

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.0/209.0 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 71.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 75.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 56.3 MB/s eta 0:00:00


In [1]:
from langchain_core.runnables import  RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_core.output_parsers import StrOutputParser
from langchain_community.graphs import Neo4jGraph
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.chat_models import ChatOllama
from langchain_experimental.graph_transformers import LLMGraphTransformer
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
from langchain_community.vectorstores import Neo4jVector
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from langchain_ollama import OllamaEmbeddings
import os
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from neo4j import  Driver

from dotenv import load_dotenv

load_dotenv()

False

setup neo4j

In [3]:
NEO4J_URI="neo4j+s://ff9635e0.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="FlpkaG2M_jHx2yxE_HkdHFAn7oU2d-ldDxM2rrlvHC8"
AURA_INSTANCEID="ff9635e0"
AURA_INSTANCENAME="Instance01"

In [3]:
NEO4J_URI="bolt://44.199.231.123"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="sharpeners-journey-hatchets"
NEO4J_DATABASE="neo4j"


In [4]:
graph = Neo4jGraph(url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD)

In [5]:
os.environ["OPENAI_API_KEY"] = ""

In [6]:
''' use gpt '''
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

llm_transformer = LLMGraphTransformer(llm=llm)

In [ ]:
'''creating graph documents by using a LLM'''
# graph_documents = llm_transformer.convert_to_graph_documents(documents)

'creating graph documents by using a LLM'

create graph documents from jsonld file

In [7]:
from langchain_community.graphs.graph_document import GraphDocument, Node, Relationship
from langchain_core.documents import Document

In [8]:
import json
with open('/Users/akshit/Documents/Projects/Python-all/information-extraction/Guidelines/Data/NCCN_prostate_4.2024_Graph_12_33.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

nodes = {}
count = 0
for item in data.get('@graph', []):
    count += 1
    # node_id = item.get('@id')
    # if node_id and node_id not in nodes:
    #     nodes[node_id] = Node(id=node_id, properties={})
print(count)

394


In [10]:
import json

# Load the JSON-LD file
with open('/Users/akshit/Documents/Projects/Python-all/information-extraction/Guidelines/Data/NCCN_prostate_4.2024_Graph_12_33.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Relationships to extract
relationship_keys = [
    # 'nccn:page-key',  ??? -> keep in properties
    'nccn:labels',
    # 'nccn:content',  ???    where to keep?
    'nccn:next',
    'nccn:prev',
    'nccn:parent',
    'nccn:reference',
    'nccn:contains'
    # 'nccn:nodelink'.  ??? -> not required
]
### PROCEED WITH THIS AND CREATE FEW GRAPH DOCUMENTS AND SEE
nodes = {}
relationships = []
graph_documents = []

# Create Node instances
for item in data.get('@graph', []):
    node_id = item.get('@id')
    metadata = { "nccn:page-key": item.get("nccn:page-key"),
                 "nccn:page-no": item.get("nccn:page-no") }
    if node_id and node_id not in nodes:
        nodes[node_id] = Node(id=node_id, properties=metadata)

# Create Relationship instances
for item in data.get('@graph', []):
    source_id = item.get('@id')
    source_node = nodes.get(source_id)
    if not source_node:
        continue
    per_node_rels = []
    for rel_type in relationship_keys:
        if rel_type in item:
            targets = item[rel_type]
            if not isinstance(targets, list):
                targets = [targets]
            for target in targets:
                # Determine target ID
                if isinstance(target, dict) and '@id' in target:
                    target_id = target['@id']
                else:
                    target_value = str(target)
                    if target_value.startswith('http://'):
                        target_id = target_value
                    else:
                        target_id = target_value
                # Create target node if it doesn't exist
                if target_id not in nodes:
                    nodes[target_id] = Node(id=target_id, properties={})
                target_node = nodes[target_id]
                # Create Relationship
                per_node_rels.append(Relationship(
                    source=source_node,
                    target=target_node,
                    type=rel_type,
                    properties={}
                ))
                relationships.extend(per_node_rels)

    # create a graph document for the node
    source_doc = Document(metadata={'source': 'NCCN_prostate_4.2024_Graph_12_33.json'}, page_content=item.get('nccn:content'))
    graph_documents.append(GraphDocument(
        nodes=[source_node],
        relationships=per_node_rels,
        source=source_doc,
    ))

# Output Nodes
node_counter = 1
node_name_map = {}
for node_id, node in nodes.items():
    node_var_name = f'node{node_counter}'
    node_name_map[node_id] = node_var_name
    print(f'{node_var_name} = {node}')
    node_counter += 1

# Output Relationships
rel_counter = 1
for rel in relationships:
    source_name = node_name_map[rel.source.id]
    target_name = node_name_map[rel.target.id]
    print(f'rel{rel_counter} = Relationship(source={source_name}, target={target_name}, type="{rel.type}", properties={{}})')
    rel_counter += 1

node1 = id='http://nccn-guideline.org/nsclc/0' properties={'nccn:page-key': 'PROS-1', 'nccn:page-no': 12}
node2 = id='http://nccn-guideline.org/nsclc/1' properties={'nccn:page-key': 'PROS-1', 'nccn:page-no': 12}
node3 = id='http://nccn-guideline.org/nsclc/2' properties={'nccn:page-key': 'PROS-1', 'nccn:page-no': 12}
node4 = id='http://nccn-guideline.org/nsclc/3' properties={'nccn:page-key': 'PROS-1', 'nccn:page-no': 12}
node5 = id='http://nccn-guideline.org/nsclc/4' properties={'nccn:page-key': 'PROS-1', 'nccn:page-no': 12}
node6 = id='http://nccn-guideline.org/nsclc/5' properties={'nccn:page-key': 'PROS-1', 'nccn:page-no': 12}
node7 = id='http://nccn-guideline.org/nsclc/6' properties={'nccn:page-key': 'PROS-1', 'nccn:page-no': 12}
node8 = id='http://nccn-guideline.org/nsclc/7' properties={'nccn:page-key': 'PROS-1', 'nccn:page-no': 12}
node9 = id='http://nccn-guideline.org/nsclc/8' properties={'nccn:page-key': 'PROS-1', 'nccn:page-no': 12}
node10 = id='http://nccn-guideline.org/nsclc/9

In [11]:
graph.add_graph_documents(graph_documents)

In [12]:
graph.refresh_schema()

In [13]:
from langchain_ollama import OllamaEmbeddings
# embeddings = OllamaEmbeddings(
#     model="mxbai-embed-large",
# )

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large"
    # With the `text-embedding-3` class
    # of models, you can specify the size
    # of the embeddings you want returned.
    # dimensions=1024
)


from langchain_community.embeddings import HuggingFaceBgeEmbeddings
# embeddings = HuggingFaceBgeEmbeddings(
#     model_name="BAAI/bge-small-en",
#     model_kwargs={'device': 'cpu'},
#     encode_kwargs={'normalize_embeddings': True}
# )
vector_index = Neo4jVector.from_existing_graph(
    embeddings,
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding",
    
    
)
vector_retriever = vector_index.as_retriever()

In [61]:
print(vector_index.similarity_search("http://nccn-guideline.org/nsclc/24"))

[]


In [66]:
# vector_index.add_documents([Document(page_content="foo")])
# docs_with_score = vector_index.similarity_search_with_score("foo")
# docs_with_score[0]
dummy = Node(id="asdasc", properties={})
target_node = Node(id="rfwewd", properties={})
dummy_rel = Relationship(source=dummy,
                    target=target_node,
                    type='label',
                    properties={}
                ) 
source_doc = Document(metadata={'source': 'NCCN_prostate_4.2024_Graph_12_33.json'}, page_content="Clinically localized prostate cancer (Any T, N0, M0 or Any T, NX, MX)")
graph_documents.append(GraphDocument(
        nodes=[dummy],
        relationships=[dummy_rel],
        source=source_doc,
    ))

In [67]:
graph.add_graph_documents([graph_documents[-1]])

[#D44B]  _: <CONNECTION> error: Failed to read from defunct connection IPv4Address(('44.199.231.123', 7687)) (ResolvedIPv4Address(('44.199.231.123', 7687))): OSError('No data')
Transaction failed and will be retried in 0.8929931356074672s (Failed to read from defunct connection IPv4Address(('44.199.231.123', 7687)) (ResolvedIPv4Address(('44.199.231.123', 7687))))


In [68]:
graph.refresh_schema()

In [71]:
vector_data = vector_retriever.invoke("Clinically localized prostate cancer (Any T, N0, M0 or Any T, NX, MX)")

In [72]:
print(vector_data)

[]


In [29]:
driver = GraphDatabase.driver(
        uri = NEO4J_URI,
        auth = (NEO4J_USERNAME,
                NEO4J_PASSWORD))

def create_fulltext_index(tx):
    query = '''
    CREATE FULLTEXT INDEX `fulltext_entity_id` 
    FOR (n:__Entity__) 
    ON EACH [n.id];
    '''
    tx.run(query)

# Function to execute the query
def create_index():
    with driver.session() as session:
        session.execute_write(create_fulltext_index)
        print("Fulltext index created successfully.")

# Call the function to create the index
try:
    create_index()
except:
    print("not created index")
    pass

# Close the driver connection
driver.close()

Fulltext index created successfully.


In [41]:
class Entities(BaseModel):
    """Identifying information about entities."""

    names: list[str] = Field(
        ...,
        description="All the person, organization, or business entities that "
        "appear in the text",
    )

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting organization and person entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)


entity_chain = llm.with_structured_output(Entities)

In [43]:
entity_chain.invoke("List all the ways in which bone imaging is performed.")

Entities(names=['bone imaging', 'bone scan', 'X-ray', 'CT scan', 'MRI', 'DEXA scan', 'ultrasound', 'PET scan', 'SPECT scan'])

In [47]:
def generate_full_text_query(input: str) -> str:
    words = [el for el in remove_lucene_chars(input).split() if el]
    if not words:
        return ""
    full_text_query = " AND ".join([f"{word}~2" for word in words])
    print(f"Generated Query: {full_text_query}")
    return full_text_query.strip()


# Fulltext index query
def graph_retriever(question: str) -> str:
    """
    Collects the neighborhood of entities mentioned
    in the question
    """
    result = ""
    entities = entity_chain.invoke(question)
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('fulltext_entity_id', $query, {limit:2})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": entity},
        )
        result += "\n".join([el['output'] for el in response])
    return result


In [37]:
vector_data = vector_retriever.invoke("http://nccn-guideline.org/nsclc/1")

In [52]:
print(graph_retriever("bone imaging"))

In [49]:
def graph_retriever(question: str) -> str:
    """
    Collects the neighborhood of entities mentioned
    in the question
    """
    result = ""
    entities = entity_chain.invoke(question)
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('fulltext_entity_id', $query, {limit:2})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": entity},
        )
        result += "\n".join([el['output'] for el in response])
    return result

In [53]:
graph_documents[0]

GraphDocument(nodes=[Node(id='http://nccn-guideline.org/nsclc/0', properties={'nccn:page-key': 'PROS-1', 'nccn:page-no': 12})], relationships=[Relationship(source=Node(id='http://nccn-guideline.org/nsclc/0', properties={'nccn:page-key': 'PROS-1', 'nccn:page-no': 12}), target=Node(id='http://nccn-guideline.org/nsclc/labels/0', properties={'nccn:page-key': None, 'nccn:page-no': None}), type='nccn:labels', properties={}), Relationship(source=Node(id='http://nccn-guideline.org/nsclc/0', properties={'nccn:page-key': 'PROS-1', 'nccn:page-no': 12}), target=Node(id='http://nccn-guideline.org/nsclc/1', properties={'nccn:page-key': 'PROS-1', 'nccn:page-no': 12}), type='nccn:next', properties={})], source=Document(metadata={'source': 'NCCN_prostate_4.2024_Graph_12_33.json', 'id': 'becb15a6522bc67b4c36866350109f2a'}, page_content='Clinically localized prostate cancer (Any T, N0, M0 or Any T, NX, MX)'))

0
